
# **Workshop Notebook: Preparing Cloud-Based Seismic & Geodetic Data for Research**  

**Duration:** ~4 hours  

**Audience:** Early-career professionals in geophysics, seismology, geodesy, and Earth data science.  

This notebook follows the workshop structure with one hands-on Python exercise per part.  
If you do not have access to cloud buckets in your environment, each exercise includes a **synthetic fallback** so you can complete it offline.

---

## Quick Setup

> You can run with your existing environment, or install recommended packages (comment/uncomment as needed).  
> If running in a managed JupyterLab (e.g., institutional hub), consult local docs.

```bash
# (optional) in a terminal:
# mamba create -n cloud-geo python=3.11 -y
# mamba activate cloud-geo
# mamba install -c conda-forge obspy numpy scipy pandas matplotlib scikit-learn pyproj h5py dask fsspec s3fs -y
# pip install pygmt==0.12.0  # optional for mapping
```

**Data options:**  
- **Option A (recommended):** Point to your cloud buckets (e.g., `s3://...`, `gs://...`) or local files.  
- **Option B:** Use the **synthetic generators** included below to complete the exercises with placeholder data.

---


In [ ]:

# Imports used throughout the workshop
import os
import io
import json
import math
import time
import glob
import random
import pathlib
from dataclasses import dataclass

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional imports guarded by try/except (so notebook still runs if missing)
try:
    from obspy import read, Stream, Trace, UTCDateTime
except Exception as e:
    read = None
    Stream = None
    Trace = None
    UTCDateTime = None
    print("ObsPy not available. Synthetic seismic fallback will be used.")

try:
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report, confusion_matrix
except Exception as e:
    RandomForestClassifier = None
    print("scikit-learn not available; ML examples will be illustrative only.")

try:
    import fsspec  # to read cloud data like s3:// or gs://
except Exception:
    fsspec = None
    print("fsspec not available; cloud file access will be skipped.")

# Plotting config
plt.rcParams["figure.figsize"] = (9, 4)
plt.rcParams["axes.grid"] = True



## Part 1 — Introduction & Context (Exercise ~30–40 min)

**Learning goals**  
- Recognize key file formats (MiniSEED, RINEX) and metadata.  
- Explore small seismic & GNSS time series and visualize them.  
- Understand how cloud URIs are accessed in Python with `fsspec` (if available).

### Exercise 1 — Quick Data Tour & Visualization
**Task:** Load a short seismic waveform and a short GNSS time series from either a cloud bucket or local files; if unavailable, generate **synthetic** data. Plot both.

**What you’ll submit:** One or two plots + 2–3 bullet points describing anything notable (e.g., noise level, sampling rate, gaps).


In [ ]:

# ==== Configuration: set paths if you have real data ====
SEISMIC_PATH = os.environ.get("SEISMIC_PATH", "")  # e.g., "s3://my-bucket/demos/event.mseed" or "./data/event.mseed"
GNSS_PATH    = os.environ.get("GNSS_PATH", "")     # e.g., "s3://my-bucket/gnss/sample_rinex.obs" or "./data/sample_rinex.csv"

# ==== Helpers ====
def read_text_with_fsspec(path):
    if fsspec is None:
        raise RuntimeError("fsspec not installed; cannot read cloud paths.")
    with fsspec.open(path, 'r') as f:
        return f.read()

def maybe_read_mseed(path):
    if not path or read is None:
        return None
    try:
        if fsspec and (path.startswith("s3://") or path.startswith("gs://")):
            with fsspec.open(path, 'rb') as f:
                return read(f)
        else:
            return read(path)
    except Exception as e:
        print("Could not read MiniSEED:", e)
        return None

def synthetic_seismic(npts=4000, dt=0.005, f0=5.0, noise=0.2):
    t = np.arange(npts) * dt
    # Ricker wavelet with noise
    pi2 = (np.pi**2)
    r = (1 - 2*pi2*(f0**2)*(t-2)**2) * np.exp(-pi2*(f0**2)*(t-2)**2)
    x = r + noise * np.random.randn(npts)
    return t, x

def synthetic_gnss_days(ndays=5, seed=42):
    rng = np.random.default_rng(seed)
    time_idx = pd.date_range("2023-01-01", periods=ndays*24, freq="H")
    # Simulate vertical displacement (mm) with diurnal cycle
    trend = np.linspace(0, 5, len(time_idx))
    diurnal = 2.0*np.sin(2*np.pi*(np.arange(len(time_idx))%24)/24.0)
    noise = rng.normal(0, 0.8, size=len(time_idx))
    disp_mm = trend + diurnal + noise
    df = pd.DataFrame({"time": time_idx, "disp_mm": disp_mm}).set_index("time")
    return df

# ==== Load or synthesize data ====
st = maybe_read_mseed(SEISMIC_PATH)
if st is None:
    print("Using synthetic seismic trace.")
    t, x = synthetic_seismic()
    seismic_df = pd.DataFrame({"t_s": t, "amp": x})
else:
    tr = st[0]
    t = np.arange(tr.stats.npts) * tr.stats.delta
    seismic_df = pd.DataFrame({"t_s": t, "amp": tr.data})

# GNSS
if GNSS_PATH and os.path.exists(GNSS_PATH):
    gnss_df = pd.read_csv(GNSS_PATH, parse_dates=["time"]).set_index("time")
else:
    print("Using synthetic GNSS time series.")
    gnss_df = synthetic_gnss_days()

# ==== Plot ====
fig, ax = plt.subplots()
ax.plot(seismic_df["t_s"], seismic_df["amp"])
ax.set_title("Seismic waveform (synthetic if no MiniSEED provided)")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude")
plt.show()

fig, ax = plt.subplots()
gnss_df["disp_mm"].plot(ax=ax)
ax.set_title("GNSS vertical displacement (mm) — synthetic if no file provided")
ax.set_ylabel("Displacement (mm)")
plt.show()

# ==== Your notes ====
print("📝 TODO: In a markdown cell, write 2–3 bullet points about what you observe.")



## Part 2 — Data Access & Preparation (Exercise ~45–60 min)

**Learning goals**  
- Ingest seismic MiniSEED and GNSS time series from cloud or local storage.  
- Apply **preprocessing**: detrending, filtering, resampling, outlier removal.  
- Persist cleaned outputs back to cloud/local (parquet/csv).

### Exercise 2 — Build a Reusable Prep Function
**Task:** Implement the TODOs to complete `prep_seismic()` and `prep_gnss()` functions, then run them on your data (or synthetic). Save results.


In [ ]:

# ==== Seismic preprocessing ====
from scipy.signal import butter, filtfilt

def butter_bandpass(low, high, fs, order=4):
    ny = 0.5 * fs
    b, a = butter(order, [low/ny, high/ny], btype='band')
    return b, a

def prep_seismic(df, fs=200.0, band=(1.0, 20.0)):
    # Detrend and bandpass filter a seismic time series DataFrame (columns: t_s, amp)
    df = df.copy()
    df["amp_dt"] = df["amp"] - df["amp"].mean()

    # Bandpass filter
    b, a = butter_bandpass(band[0], band[1], fs)
    try:
        df["amp_filt"] = filtfilt(b, a, df["amp_dt"])
    except Exception as e:
        print("Filter failed (try adjusting band or fs). Using detrended only.", e)
        df["amp_filt"] = df["amp_dt"]
    return df[["t_s", "amp_filt"]]

# ==== GNSS preprocessing ====
def hampel_filter(series, k=5, t0=3.0):
    # Simple Hampel outlier filter.
    n = len(series)
    new_series = series.copy().astype(float)
    L = 1.4826
    for i in range(n):
        i1 = max(i-k, 0)
        i2 = min(i+k, n-1)
        window = series.iloc[i1:i2+1]
        median = window.median()
        mad = L * np.median(np.abs(window - median))
        if mad == 0:
            continue
        if np.abs(series.iloc[i] - median) > t0 * mad:
            new_series.iloc[i] = median
    return new_series

def prep_gnss(df):
    # Clean GNSS displacement: Hampel outliers, linear detrend, hourly resample.
    y = df["disp_mm"]
    y_clean = hampel_filter(y, k=6, t0=3.5)
    x = np.arange(len(y_clean))
    coeffs = np.polyfit(x, y_clean.values, 1)
    trend = np.polyval(coeffs, x)
    detrended = y_clean.values - trend
    out = pd.DataFrame({"disp_mm_clean": detrended}, index=df.index)
    out = out.resample("1H").mean().interpolate()
    return out

# ==== Apply to our data ====
# Seismic fs estimation if synthetic:
if len(seismic_df) > 1:
    dt = np.mean(np.diff(seismic_df["t_s"]))
    fs = 1.0 / dt
else:
    fs = 200.0

seis_clean = prep_seismic(seismic_df, fs=fs, band=(1.0, 20.0))
gnss_clean = prep_gnss(gnss_df)

# ==== Save to disk (you can change to cloud URIs if fsspec available) ====
outdir = pathlib.Path("./prep_outputs")
outdir.mkdir(exist_ok=True, parents=True)
seis_clean.to_parquet(outdir / "seismic_clean.parquet")
gnss_clean.to_parquet(outdir / "gnss_clean.parquet")

print("Saved cleaned outputs to:", outdir.resolve())

# ==== Visualize results ====
fig, ax = plt.subplots()
ax.plot(seis_clean["t_s"], seis_clean["amp_filt"])
ax.set_title("Seismic (cleaned)")
ax.set_xlabel("Time (s)"); ax.set_ylabel("Amplitude")
plt.show()

fig, ax = plt.subplots()
gnss_clean["disp_mm_clean"].plot(ax=ax)
ax.set_title("GNSS (cleaned, detrended)")
ax.set_ylabel("Displacement (mm, detrended)")
plt.show()

# 📝 TODO: Note any parameter choices you changed and why.



## Part 3 — Cloud-Based Analysis & ML Integration (Exercise ~60–75 min)

**Learning goals**  
- Build a minimal ML pipeline for **seismic event detection** (binary classification) with synthetic/semi-real features.  
- (Optional) Estimate an atmospheric proxy (e.g., PWV surrogate) from GNSS using simple regression as a warm-up.

### Exercise 3A — Seismic Event Classifier (Random Forest)
**Task:** Create features from short seismic windows and train a classifier to distinguish **event** vs **noise**.  
If you lack labeled data, we will **synthesize** events (Ricker wavelets) + noise.

**Deliverables:**  
- Confusion matrix + classification report  
- A brief note: which features mattered and how you would improve this for real data


In [ ]:

# ==== Feature engineering for seismic snippets ====
def window_features(x, fs):
    # Extract simple features from a 1D window x.
    feats = {}
    feats["rms"] = np.sqrt(np.mean(x**2))
    feats["peak"] = np.max(np.abs(x))
    feats["zcr"] = np.mean(np.abs(np.diff(np.sign(x))))  # zero crossing rate (approx)
    # spectral centroid proxy
    X = np.fft.rfft(x)
    freqs = np.fft.rfftfreq(len(x), d=1.0/fs)
    ps = np.abs(X)**2
    if ps.sum() > 0:
        feats["spec_centroid"] = (freqs * ps).sum() / ps.sum()
    else:
        feats["spec_centroid"] = 0.0
    return feats

def make_dataset_from_series(seis, fs, win_s=2.0, step_s=1.0, p_event=0.3, seed=0):
    # Synthesize labels: inject Ricker-like 'events' in random windows for demo purposes.
    rng = np.random.default_rng(seed)
    x = seis["amp_filt"].values
    n = len(x)
    win = int(win_s * fs)
    step = int(step_s * fs)
    X, y = [], []
    i = 0
    while i + win <= n:
        seg = x[i:i+win].copy()
        # Randomly inject an 'event'
        is_event = rng.random() < p_event
        if is_event:
            t = np.arange(win)/fs
            f0 = rng.uniform(3.0, 8.0)
            r = (1 - 2*(np.pi**2)*(f0**2)*(t-0.5)**2)*np.exp(-(np.pi**2)*(f0**2)*(t-0.5)**2)
            seg = seg + r * rng.uniform(1.5, 3.0)
        feats = window_features(seg, fs)
        X.append([feats[k] for k in ["rms","peak","zcr","spec_centroid"]])
        y.append(1 if is_event else 0)
        i += step
    return np.array(X), np.array(y)

# Build dataset
X, y = make_dataset_from_series(seis_clean, fs=fs, win_s=2.0, step_s=1.0, p_event=0.35, seed=42)
print("Features shape:", X.shape, "Labels:", y.shape, "Positives:", y.sum())

if RandomForestClassifier is None:
    print("scikit-learn not available; skipping training.")
else:
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, random_state=0, stratify=y)
    clf = RandomForestClassifier(n_estimators=150, random_state=0)
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)

    print("Classification report:")
    print(classification_report(y_te, y_pred))

    print("Confusion matrix:")
    print(confusion_matrix(y_te, y_pred))

    # Simple feature importance
    importances = clf.feature_importances_
    feat_names = ["rms","peak","zcr","spec_centroid"]
    fig, ax = plt.subplots()
    ax.bar(range(len(importances)), importances)
    ax.set_xticks(range(len(importances)))
    ax.set_xticklabels(feat_names, rotation=0)
    ax.set_title("Feature importance (RandomForest)")
    plt.show()

# 📝 TODO: Note how you would obtain *real* labels (e.g., picks/catalog) and extend features.



### Exercise 3B (Optional) — GNSS Atmospheric Proxy (Simple Regression)

**Task:** Using the cleaned GNSS time series, create a **toy surrogate** for PWV (e.g., linear combo of detrended displacement and diurnal index) and fit a regression model.  
This is illustrative; in production you would use proper GNSS meteorology products.

**Deliverables:**  
- Train/test score  
- A plot of predicted vs. target


In [ ]:

# Create a toy target (PWV surrogate) from GNSS trends + diurnal component
rng = np.random.default_rng(0)
idx = gnss_clean.index
h = idx.hour.values
diurnal = np.sin(2*np.pi*h/24.0)
noise = rng.normal(0, 0.3, size=len(idx))
target = 0.7*gnss_clean["disp_mm_clean"].values + 0.8*diurnal + noise

X = np.c_[gnss_clean["disp_mm_clean"].values, diurnal]
y = target

if RandomForestClassifier is None:
    print("scikit-learn not available; showing scatter only.")
    plt.scatter(X[:,0], y, s=10)
    plt.title("PWV surrogate vs GNSS detrended displacement")
    plt.xlabel("disp_mm_clean"); plt.ylabel("PWV surrogate")
    plt.show()
else:
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import r2_score, mean_squared_error
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, random_state=0)
    regr = RandomForestRegressor(n_estimators=200, random_state=0)
    regr.fit(X_tr, y_tr)
    y_hat = regr.predict(X_te)
    print("R^2:", r2_score(y_te, y_hat))
    print("RMSE:", np.sqrt(mean_squared_error(y_te, y_hat)))

    # Plot predicted vs target
    plt.scatter(y_te, y_hat, s=12)
    plt.xlabel("Target (surrogate)"); plt.ylabel("Prediction")
    plt.title("GNSS PWV surrogate — predicted vs. target")
    plt.plot([y_te.min(), y_te.max()], [y_te.min(), y_te.max()])
    plt.show()

# 📝 TODO: List the *real* inputs you would use for PWV (e.g., ZTD + met data) and references/tools.



## Part 4 — Reproducibility, Collaboration & Next Steps (Exercise ~30–45 min)

**Learning goals**  
- Capture parameters and environment for reproducibility.  
- Save intermediate artifacts and a minimal **pipeline** definition.  
- Discuss FAIR/ethics/licensing considerations.

### Exercise 4 — Make It Reproducible
**Task:**  
1) Save your key parameters to a JSON **run config**.  
2) Package your cleaned outputs and model artifacts into a versioned folder.  
3) (Optional) Write a minimal `environment.yml` or `requirements.txt` and a tiny `Makefile` to run the pipeline.

**Deliverables:**  
- `runs/run_YYYYmmdd_HHMM/config.json`  
- Saved cleaned parquet files (already in `prep_outputs/`) and any trained models.  
- (Optional) `requirements.txt` and `Makefile` below.


In [ ]:

import json, pathlib, shutil
from datetime import datetime

run_id = datetime.now().strftime("%Y%m%d_%H%M")
run_dir = pathlib.Path("runs") / f"run_{run_id}"
run_dir.mkdir(parents=True, exist_ok=True)

config = {
    "seismic": {"band": [1.0, 20.0], "fs": float(fs)},
    "gnss": {"resample": "1H", "outlier_filter": "Hampel(k=6,t0=3.5)"},
    "ml": {"seis_win_s": 2.0, "seis_step_s": 1.0, "rf_n_estimators": 150}
}

with open(run_dir / "config.json", "w") as f:
    json.dump(config, f, indent=2)

# Copy artifacts if present
artifacts = ["prep_outputs/seismic_clean.parquet", "prep_outputs/gnss_clean.parquet"]
for a in artifacts:
    p = pathlib.Path(a)
    if p.exists():
        shutil.copy(p, run_dir / p.name)

print("Saved run config & artifacts to:", run_dir.resolve())



### Optional: Minimal `requirements.txt`, `Makefile`, & Next Steps

You can place these in your repo to standardize runs.

**requirements.txt**
```
obspy
numpy
scipy
pandas
matplotlib
scikit-learn
pyproj
fsspec
s3fs
```

**Makefile**
```
run:
	python -m jupyter nbconvert --to notebook --execute Cloud_Seismo_Geodesy_Workshop.ipynb --inplace
```

**Next steps / discussion prompts**
- What metadata would you store with each run (sensor IDs, event IDs, GNSS station list)?  
- How would you align/merge seismic picks with catalogs to build **real** labels?  
- Which cloud services (S3/GS/Azure) will you target, and what IAM/permissions are required?  
- How will you implement FAIR principles & licensing in your project?  



---

## Wrap-up & Reflection

**📝 Prompt:** In 4–6 bullet points, summarize your key takeaways and note one improvement you would make to:  
- your preprocessing choices  
- your ML features/labels  
- your reproducibility setup
